<a href="https://colab.research.google.com/github/SEDHUMADHAVAN-CYBER/Multi_Agentx-Langgraph-langchain/blob/main/Multi__Agentx2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio langgraph langchain-groq -q
print("✅ Gradio ready!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.8/495.8 kB 13.7 MB/s eta 0:00:00
✅ Gradio ready!


In [ ]:
import gradio as gr
from typing import TypedDict, List, Optional
from langchain_groq import ChatGroq
from google.colab import userdata
import random

class AgentState(TypedDict, total=False):
    user_input: str
    plan: List[str]
    collaborator_output: Optional[str]
    competitor_output: Optional[str]
    negotiated_output: Optional[str]
    final_output: Optional[str]
    reward_score: Optional[int]

# LLM Setup
llm = ChatGroq(
    groq_api_key=userdata.get("GROQ_API_KEY"),
    model_name="llama-3.1-8b-instant"
)

# Pure agent functions
def planner_agent(state: AgentState) -> AgentState:
    prompt = f"Break this task into 3 clear steps:\n{state['user_input']}"
    response = llm.invoke(prompt)
    steps = [s.strip("- ").strip() for s in response.content.split("\n") if s.strip()]
    if not steps: steps = ["Analyze", "Explain", "Conclude"]
    return {"plan": steps[:3]}

def collaborator_agent(state: AgentState) -> AgentState:
    prompt = f"Task: {state['user_input']}\nPlan: {state['plan']}\nProvide helpful, optimistic solution."
    response = llm.invoke(prompt)
    return {"collaborator_output": response.content}

def competitor_agent(state: AgentState) -> AgentState:
    prompt = f"Task: {state['user_input']}\nCritique collaborator. Different perspective."
    response = llm.invoke(prompt)
    return {"competitor_output": response.content}

def negotiation_agent(state: AgentState) -> AgentState:
    prompt = f"Collaborator: {state['collaborator_output'][:1000]}\nCompetitor: {state['competitor_output'][:1000]}\nCombine into stronger answer."
    response = llm.invoke(prompt)
    return {"negotiated_output": response.content}

def judge_agent(state: AgentState) -> AgentState:
    prompt = f"Task: {state['user_input']}\nCollaborator: {state['collaborator_output'][:800]}\nCompetitor: {state['competitor_output'][:800]}\nDecide which is better."
    response = llm.invoke(prompt)
    score = random.randint(1, 10)
    return {"final_output": response.content, "reward_score": score}

# Build Graph
from langgraph.graph import StateGraph, END
graph = StateGraph(AgentState)
graph.add_node("planner", planner_agent)
graph.add_node("collaborator", collaborator_agent)
graph.add_node("competitor", competitor_agent)
graph.add_node("negotiator", negotiation_agent)
graph.add_node("judge", judge_agent)

graph.set_entry_point("planner")
graph.add_edge("planner", "collaborator")
graph.add_edge("planner", "competitor")
graph.add_edge("collaborator", "negotiator")
graph.add_edge("competitor", "negotiator")
graph.add_edge("negotiator", "judge")
graph.add_edge("judge", END)

multi_agent_graph = graph.compile()

# MAIN FUNCTION
def run_multi_agent(question):
    state = {"user_input": question}
    final_state = multi_agent_graph.invoke(state)

    # Format output
    plan = ", ".join(final_state.get('plan', []))
    collab = final_state.get('collaborator_output', 'N/A')[:500] + "..."
    comp = final_state.get('competitor_output', 'N/A')[:500] + "..."
    neg = final_state.get('negotiated_output', 'N/A')[:500] + "..."
    judgment = final_state.get('final_output', 'N/A')
    score = final_state.get('reward_score', 0)

    return f"""
🤖 **Multi-Agent Demo Results**

**🧠 Plan:** {plan}

**🤝 Collaborator:** {collab}
**⚔️ Competitor:** {comp}

**💬 Negotiated:** {neg}

**🏆 Judgment:** {judgment}

🎉 **Score: {score}/10** ⭐⭐⭐
    """

# Gradio Interface
with gr.Blocks(title="Multi-Agent Judge Demo", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 Multi-Agent Collaboration System")
    gr.Markdown("**Planner → Collaborator vs Competitor → Negotiator → Judge**")

    with gr.Row():
        with gr.Column(scale=1):
            input_box = gr.Textbox(
                label="Ask the Multi-Agent Team",
                placeholder="What is machine learning?",
                lines=2
            )
        with gr.Column(scale=2):
            run_btn = gr.Button("🚀 Run Agents", variant="primary", size="lg")

    output = gr.Markdown("**Results appear here...**")

    gr.Examples(
        examples=[
            "What is machine learning?",
            "Explain neural networks",
            "How does blockchain work?"
        ],
        inputs=input_box
    )

    run_btn.click(run_multi_agent, inputs=input_box, outputs=output)

# LAUNCH PUBLICLY
demo.launch(share=True, debug=True)


/tmp/ipython-input-1888275100.py:99: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Multi-Agent Judge Demo", theme=gr.themes.Soft()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5ed6fb4aead4e43fab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
